<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/logic/statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
DBLink = "https://cloud-project-5adfc-default-rtdb.europe-west1.firebasedatabase.app/"

#Imports
import paho.mqtt.client as mqtt
import json
import threading

# Callback function for when a message is received
def indoor_on_message(client, userdata, msg):
  try:
    data = json.loads(msg.payload.decode())
    # Extract values with lowercase field names
    temperature = data.get("Temperature", "N/A")
    humidity = data.get("Humidity", "N/A")
    pressure = data.get("Pressure", "N/A")
    distance = data.get("Distance", "N/A")
    print(f"Indoor: Temperature: {temperature}°C, Humidity: {humidity}%, Pressure: {pressure} hPa, Distance: {distance} mm")

  except json.JSONDecodeError:
    print("Received invalid JSON data")

# Callback function for when a message is received
def outdoor_on_message(client, userdata, msg):
  try:
    data = json.loads(msg.payload.decode())
    # Extract values with lowercase field names
    temperature = data.get("Temperature", "N/A")
    humidity = data.get("Humidity", "N/A")
    dlight = data.get("Dlight", "N/A")
    print(f"Outdoor: Temperature: {temperature}°C, Humidity: {humidity}%, Dlight: {dlight} Lux")

  except json.JSONDecodeError:
    print("Received invalid JSON data")

# Get the data and print

# MQTT setup
broker = "test.mosquitto.org"
topic_indoor = "braude/D106/indoor"
topic_outdoor = "braude/D106/outdoor"

client_indoor = mqtt.Client()
client_indoor.on_message = indoor_on_message

client_indoor.connect(broker, 1883, 60)
client_indoor.subscribe(topic_indoor)

client_outdoor = mqtt.Client()
client_outdoor.on_message = outdoor_on_message

client_outdoor.connect(broker, 1883, 60)
client_outdoor.subscribe(topic_outdoor)

# Create and start threads for each client
indoor_thread = threading.Thread(target=client_indoor.loop_forever)
outdoor_thread = threading.Thread(target=client_outdoor.loop_forever)

indoor_thread.start()
outdoor_thread.start()

# Keep the main thread alive
outdoor_thread.join()
indoor_thread.join()

from firebase import firebase

FBconn = firebase.FirebaseApplication(DBLink, None)

# Read the Excel file into a pandas DataFrame
df = pd.read_excel('sensor_data.xlsx')

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Use the Timestamp as the key for Firebase
    timestamp_str = row['Timestamp'].strftime('%Y-%m-%d %H:%M:%S') # Format timestamp as string

    # Create a dictionary with the sensor data for this timestamp
    sensor_data = {
        'Outdoor Dlight': row['Outdoor Dlight'],
        'Outdoor Temperature': row['Outdoor Temperature'],
        'Outdoor Humidity': row['Outdoor Humidity'],
        'Indoor Humidity': row['Indoor Humidity'],
        'Indoor Temperature': row['Indoor Temperature'],
        'Indoor Pressure': row['Indoor Pressure'],
        'Indoor Distance': row['Indoor Distance']
    }

    # Save the data to Firebase with the timestamp as the key
    FBconn.put('/sensor_readings', timestamp_str, sensor_data)

    print(f"Saved data for timestamp: {timestamp_str}")

import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta

# Function to fetch data from Firebase for a specific time range
def fetch_data_from_firebase(full_start_time_str):
    try:
        start_time = datetime.strptime(full_start_time_str, '%Y-%m-%d %H:%M:%S')
    except ValueError as e:
        print(f"Error parsing start time string '{full_start_time_str}': {e}")
        return pd.DataFrame()

    end_time = start_time + timedelta(hours=1)

    all_data = FBconn.get('/sensor_readings', None)

    if not all_data:
        return pd.DataFrame()

    filtered_data = {}
    for timestamp_str, data in all_data.items():
        try:
            timestamp = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S')
            if start_time <= timestamp < end_time:
                filtered_data[timestamp] = data
        except ValueError:
            print(f"Skipping entry with invalid timestamp in Firebase: {timestamp_str}")
            pass

    if not filtered_data:
        return pd.DataFrame()

    df = pd.DataFrame.from_dict(filtered_data, orient='index')
    if not df.empty: # Ensure index is DatetimeIndex for plotting if df is not empty
        df.index = pd.to_datetime(df.index)
        df.index.name = 'Timestamp'
        df = df.sort_index()
    return df

# Helper function to create plots (reduces redundancy)
def create_sensor_plot(df, column_name, title, ylabel):
    plt.figure(figsize=(10, 4))
    if column_name in df.columns and not df[column_name].empty:
        plt.plot(df.index, df[column_name])
    else:
        plt.text(0.5, 0.5, f'No data for {column_name}', horizontalalignment='center', verticalalignment='center')
    plt.grid(True) # Add grid
    plt.title(title)
    plt.xlabel('Timestamp')
    plt.ylabel(ylabel)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plot_fig = plt.gcf()
    plt.close()
    return plot_fig

# Function to generate plots for the selected time range
def generate_plots(selected_date_str, selected_hour_str):
    if not selected_date_str or not selected_hour_str:
        print("Date or hour not selected.")
        # Return Nones for all 7 plot outputs
        return None, None, None, None, None, None, None

    try:
        # selected_date_str is already "YYYY-MM-DD"
        selected_hour_int = int(selected_hour_str)
        full_start_time_str = f"{selected_date_str} {selected_hour_int:02d}:00:00"
    except ValueError:
        print(f"Invalid hour format: {selected_hour_str}")
        return None, None, None, None, None, None, None

    df_hour = fetch_data_from_firebase(full_start_time_str)

    if df_hour.empty:
        print(f"No data found for {full_start_time_str}")
        return None, None, None, None, None, None, None

    plot_dlight = create_sensor_plot(df_hour, 'Outdoor Dlight', 'Outdoor Dlight', 'Dlight (Lux)')
    plot_out_temp = create_sensor_plot(df_hour, 'Outdoor Temperature', 'Outdoor Temperature', 'Temp (°C)')
    plot_out_humidity = create_sensor_plot(df_hour, 'Outdoor Humidity', 'Outdoor Humidity', 'Humidity (%)')
    plot_in_humidity = create_sensor_plot(df_hour, 'Indoor Humidity', 'Indoor Humidity', 'Humidity (%)')
    plot_in_temp = create_sensor_plot(df_hour, 'Indoor Temperature', 'Indoor Temperature', 'Temp (°C)')
    plot_pressure = create_sensor_plot(df_hour, 'Indoor Pressure', 'Indoor Pressure', 'Pressure (hPa)')
    plot_distance = create_sensor_plot(df_hour, 'Indoor Distance', 'Indoor Distance', 'Distance (mm)')

    return plot_dlight, plot_out_temp, plot_out_humidity, plot_in_humidity, plot_in_temp, plot_pressure, plot_distance

# --- Data Preparation for Dropdowns ---
all_data_keys = FBconn.get('/sensor_readings', None) #
if all_data_keys:
    available_timestamps_str = sorted(all_data_keys.keys())
else:
    available_timestamps_str = []

# Get unique dates as STRINGS
unique_date_strings = sorted(list(set([
    datetime.strptime(ts, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d')
    for ts in available_timestamps_str
])))

#ToDo: Add refresh button to be able to see new datetime if running too long    ##############################################

# Create mapping from STRING date to list of hours (as strings or ints)
date_to_hours_map = {}
for ts_str in available_timestamps_str:
    dt_obj = datetime.strptime(ts_str, '%Y-%m-%d %H:%M:%S')
    date_key_str = dt_obj.strftime('%Y-%m-%d')
    hour_val = dt_obj.hour # Keep as int for now, or str(dt_obj.hour)
    if date_key_str not in date_to_hours_map:
        date_to_hours_map[date_key_str] = []
    if hour_val not in date_to_hours_map[date_key_str]: # Avoid duplicate hours
        date_to_hours_map[date_key_str].append(hour_val)

for date_key_str in date_to_hours_map:
    date_to_hours_map[date_key_str].sort()
    # Convert hours to string if you prefer string choices for hour dropdown too
    date_to_hours_map[date_key_str] = [str(h) for h in date_to_hours_map[date_key_str]]

def update_hour_dropdown_and_plots(selected_date_str_event):
    # This function is triggered by date_dropdown change.
    # It needs to update hour_dropdown's choices AND trigger plot regeneration.
    new_hour_choices = date_to_hours_map.get(selected_date_str_event, [])
    new_selected_hour = new_hour_choices[0] if new_hour_choices else None

    # Update plots based on the new date and the first available hour
    plot_updates = generate_plots(selected_date_str_event, new_selected_hour)

    # Return updates for time_dropdown and all plot outputs
    return [gr.update(choices=new_hour_choices, value=new_selected_hour)] + list(plot_updates if plot_updates else [None]*7)